# Build a House Price Predictor with SAP AI Core
Author: https://github.com/dhrubpaul

You need Docker to complete this tutorial. If you are running this Jupyter notebook on web, we recommend to use your local system along with it.

The steps are analogous with the tutorial: https://developers.sap.com/tutorials/ai-core-code.html
Please open the tutorial and this notebook side-by-side for better understanding.

## Pre-requisite 
Create connection with SAP AI Core. Edit the below cell

In [1]:
%pip install generative-ai-hub-sdk

     |████████████████████████████████| 265 kB 1.6 MB/s eta 0:00:01
     |████████████████████████████████| 326 kB 11.7 MB/s eta 0:00:01
     |████████████████████████████████| 409 kB 11.0 MB/s eta 0:00:01
     |████████████████████████████████| 86 kB 6.5 MB/s  eta 0:00:01
     |████████████████████████████████| 78 kB 3.6 MB/s eta 0:00:011
  Using cached sniffio-1.3.1-py3-none-any.whl (10 kB)
  Using cached httpx-0.27.0-py3-none-any.whl (75 kB)
     |████████████████████████████████| 2.0 MB 10.5 MB/s eta 0:00:01
  Using cached httpcore-1.0.5-py3-none-any.whl (77 kB)
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import yaml
# Load Library
from ai_core_sdk.ai_core_v2_client import AICoreV2Client

# Load environment variables from credentials.yaml
with open('credentials.yaml', 'r') as file:
    env_vars = yaml.safe_load(file)

os.environ['AICORE_CLIENT_ID'] = env_vars['AICORE_CLIENT_ID']
os.environ['AICORE_CLIENT_SECRET'] = env_vars['AICORE_CLIENT_SECRET']
os.environ['AICORE_RESOURCE_GROUP'] = env_vars['AICORE_RESOURCE_GROUP']
os.environ['AICORE_AUTH_URL'] = env_vars['AICORE_AUTH_URL']
os.environ['AICORE_BASE_URL'] = env_vars['AICORE_BASE_URL']

# Create Connection using environment variables
ai_core_client = AICoreV2Client(
    base_url=os.environ['AICORE_BASE_URL'] + "/v2",
    auth_url=os.environ['AICORE_AUTH_URL'] + "/oauth/token",
    client_id=os.environ['AICORE_CLIENT_ID'],
    client_secret=os.environ['AICORE_CLIENT_SECRET']
)
# no output is expected

## Step 1: Create a Docker account and generate a Docker access token

*Create a repository on hub.docker.com . You may use any your own Docker registry.* Refere step 1 of the tutorial: https://developers.sap.com/tutorials/ai-core-code.html#cf7b33ab-c455-47ee-a812-33a1ff587cf0

## Step 2: Install Docker Desktop

*Download and install docker https://www.docker.com/products/docker-desktop/*

## Step 3: Write AI code for house price prediction

Please refer the step 3 of the tutorial: https://developers.sap.com/tutorials/ai-core-code.html#3730e9e1-15d5-4cd3-a4f7-27a82d2297da

## Step 4: Transform your AI code into a Docker image

*Please refer to step 4: https://developers.sap.com/tutorials/ai-core-code.html#61ada9bc-8a9a-4671-8e92-fde71c78a8f1*

## Step 5: Connect your local system to a Docker account

*Please refer to step 5: https://developers.sap.com/tutorials/ai-core-code.html#beb0c055-7441-41d3-a285-304a1c41b6fb*

## Step 6: Upload a Docker image to the cloud

*Please refer to step 6: https://developers.sap.com/tutorials/ai-core-code.html#2aa8d2f5-9101-4e66-8403-b58e15ee18c5*

## Step 7: Store your Docker credentials in SAP AI Core as a Docker registry secret

In [3]:
import yaml

# Load the YAML file
with open('credentials.yaml', 'r') as file:
    config = yaml.safe_load(file)

# Extract the necessary values
docker_registry_url = config['DOCKER_TOKEN_NAME']
docker_username = config['DOCKER_USERNAME']
docker_access_token = config['DOCKER_ACCESS_TOKEN']

# Use the extracted values in your snippet
response = ai_core_client.docker_registry_secrets.create(
    name="credstutorialrepo",
    data={
        ".dockerconfigjson": f'{{"auths":{{"{docker_registry_url}":{{"username":"{docker_username}","password":"{docker_access_token}"}}}}}}'
    }
)

print(response.__dict__)


{'message': 'secret has been created'}


## Step 8: Reference a Docker image in your AI workflow


*Please refer step 8 of the tutorial: https://developers.sap.com/tutorials/ai-core-code.html#d5976faf-77b3-4c8d-a52f-82b78862386f*

## Step 9: Check the sync status of your workflow

In [ ]:
# Get application status
response = ai_core_client.applications.get_status(application_name='learning-scenarios-app')

print(response.message)
for sync_status in response.sync_ressources_status:
    print(sync_status.__dict__)

- should see `status: 'Synced'`

In [ ]:
# List available executables
response = ai_core_client.executable.query(
    scenario_id = "learning-code", resource_group='default'
)

for executable in response.resources:
    print(executable.__dict__)


- should see `id: code-pipeline`. The value is derived from your workflow from the annotation `name`.

## Step 10: Run your AI workflow

In [ ]:
# Create configuration
response = ai_core_client.configuration.create(
    name = "code-workflow-1",
    scenario_id = "learning-code",
    executable_id = "code-pipeline",
    resource_group = "default"
)
print(response.__dict__)


In [ ]:
# Create and start execution
response = ai_core_client.execution.create(
    configuration_id = '<YOUR_CONFIGURATION_ID>',
    resource_group = 'default'
)

response.__dict__

In [ ]:
# Get execution status from SAP AI Core
# execute this multiple times in interval of 30 seconds
response = ai_core_client.execution.get(
    execution_id = '<YOUR_EXECUTION_ID', # Change this
    resource_group = 'default'
)

response.__dict__


- should end up with **Status: DEAD**

## Step 11 Look for error logs in execution


In [ ]:
# get execution logs
response = ai_core_client.execution.query_logs(
    execution_id = '<YOUR_EXECUTION_ID>',
    resource_group = 'default',
    start = datetime(1990, 1, 1) # Optional, required when execution is older than one hour
)

for log in response.data.result:
    print(log.__dict__)


## Step 12: Fix and update code

*Refer step 12 of the tutorial: https://developers.sap.com/tutorials/ai-core-code.html#d3cdbc12-922d-48b2-a197-44f0d8fbd867*

## Step 13: Update your AI workflow

*Refer step 13 of the tutorial: https://developers.sap.com/tutorials/ai-core-code.html#3fedf413-00e5-4554-85bc-06130758678a*

## Step 14: Create Execution

In [ ]:
# Create execution
response = ai_core_client.executable.query(
    scenario_id = "learning-code", resource_group='default'
)

for executable in response.resources:
    print(executable.__dict__)


In [ ]:
# Get execution status from SAP AI Core
# execute this multiple times in interval of 30 seconds
response = ai_core_client.execution.get(
    execution_id = '<YOUR_EXECUTION_ID', # Change this
    resource_group = 'default'
)

response.__dict__


- should show **Status: COMPLETED**

In [ ]:
# Get Execution Logs
response = ai_core_client.execution.query_logs(
    execution_id = '<YOUR_EXECUTION_ID>',
    resource_group = 'default'
)

for log in response.data.result:
    print(log.__dict__)
